In [1]:
import requests
import json
import elevenlabs
from openai import OpenAI
import time

In [2]:
with open('../credentials.json', 'r') as file:
    cred = json.load(file)

elevenlabs.set_api_key(cred["elevenlabs_api_key"])
client = OpenAI(api_key=cred["openai_api_key"])

assistant_id = "asst_nv9sFR2LXHefrpwws4EUT6xU"
voice_id = "vDxT1hhO5CjgzUI7e2vF"

my_assistant = client.beta.assistants.retrieve(assistant_id)

new_thread = client.beta.threads.create()

In [8]:
def new_question(question_text):
  message_request = client.beta.threads.messages.create(
      thread_id = new_thread.id,
      role = "user",
      content = question_text
  )

  run = client.beta.threads.runs.create(
    thread_id = message_request.thread_id,
    assistant_id = my_assistant.id
  )

  run = client.beta.threads.runs.retrieve(
    thread_id = message_request.thread_id,
    run_id = run.id
  )

  def wait_for_completed():
      while True:
          response = client.beta.threads.runs.retrieve(thread_id = new_thread.id, run_id = run.id)
          if response.status == "completed":
              return response
          time.sleep(1)

  run_return = wait_for_completed()

  messages = client.beta.threads.messages.list(
    thread_id = message_request.thread_id
  )

  audio = elevenlabs.generate(
      text = messages.data[0].content[0].text.value,
          voice = voice_id
  )

  elevenlabs.play(audio)
  elevenlabs.save(audio, "test.wav")

In [9]:
new_question("What will I make of my life?")